<a href="https://colab.research.google.com/github/zinseo964/dam_flow/blob/main/usingLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
import tensorflow as tf
from keras.layers import Dense, LSTM

In [12]:
from google.colab import drive
drive.mount('/content/drive')

rain_data211 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dam_flow_csv/rn_20240130172151.csv',encoding='cp949')
rain_data101 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dam_flow_csv/rn_20240130172202.csv',encoding='cp949')
rain_data100 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dam_flow_csv/rn_20240130172215.csv',encoding='cp949')
rain_data212 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dam_flow_csv/rn_20240130172236.csv',encoding='cp949')
rain_data90 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dam_flow_csv/rn_20240130172245.csv',encoding='cp949')

# rain_data = pd.merge(rain_data211, rain_data101, how='outer')
from functools import reduce

rain_data = reduce(lambda x,y : pd.merge(x,y, on='날짜', how='outer'),[rain_data211, rain_data101, rain_data100, rain_data212, rain_data90])

rain_col = [i for i in rain_data.columns if '강수' in i]
rain_data[rain_col] = rain_data[rain_col].fillna(0)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
dam_flow = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dam_flow_csv/only_dam_flow.csv',encoding='utf-8')

df = reduce(lambda x,y : pd.merge(x,y, on='날짜', how='outer'),[rain_data, dam_flow])

In [14]:
df['211강수량(mm)'] = df['211강수량(mm)'].astype(float)
df['101강수량(mm)'] = df['101강수량(mm)'].astype(float)
df['100강수량(mm)'] = df['100강수량(mm)'].astype(float)
df['212강수량(mm)'] = df['212강수량(mm)'].astype(float)
df['90강수량(mm)'] = df['90강수량(mm)'].astype(float)

In [15]:
X = df[['날짜', '211강수량(mm)', '101강수량(mm)', '100강수량(mm)', '212강수량(mm)', '90강수량(mm)']]
X['날짜'] = pd.to_datetime(X['날짜'])
X['year'] = X['날짜'].dt.year
X['month'] = X['날짜'].dt.month
X['day'] = X['날짜'].dt.day
X = X.drop('날짜', axis=1)

<ipython-input-15-6537a5937448>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['날짜'] = pd.to_datetime(X['날짜'])


In [16]:
X_for_scaler = X.drop(['year', 'month', 'day'], axis=1)  # year, month, day, hour 열 제외
scaler = MinMaxScaler(feature_range=(0, 1))
X_for_scaler = scaler.fit_transform(X_for_scaler)

In [17]:
X[['211강수량(mm)', '101강수량(mm)', '100강수량(mm)', '212강수량(mm)', '90강수량(mm)']] = X_for_scaler
Y = df[['유입량(㎥/s)']]

print(X)

      211강수량(mm)  101강수량(mm)  100강수량(mm)  212강수량(mm)  90강수량(mm)  year  month  \
0       0.000000    0.000000    0.000000    0.000000   0.000000  2009      1   
1       0.000000    0.000000    0.000000    0.000000   0.000000  2009      1   
2       0.000000    0.000000    0.000000    0.000000   0.000000  2009      1   
3       0.000000    0.000000    0.000000    0.000000   0.000000  2009      1   
4       0.000000    0.000000    0.000000    0.000000   0.000000  2009      1   
...          ...         ...         ...         ...        ...   ...    ...   
5473    0.000000    0.000000    0.000000    0.000000   0.000000  2023     12   
5474    0.000000    0.000000    0.000000    0.000000   0.000000  2023     12   
5475    0.000000    0.000000    0.000000    0.000000   0.000000  2023     12   
5476    0.026984    0.037714    0.011414    0.037037   0.001899  2023     12   
5477    0.012302    0.020190    0.029777    0.021351   0.072688  2023     12   

      day  
0       1  
1       2  
2  

In [18]:
model = Sequential()
model.add(LSTM(128, input_shape=(X.shape[1], 1)))
model.add(Dense(1))

In [19]:
# 모델 컴파일
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

# 모델 학습
model.fit(X, Y, epochs=50, batch_size=1, verbose=2)

Epoch 1/50
5478/5478 - 46s - loss: 40269.5391 - accuracy: 0.0000e+00 - 46s/epoch - 8ms/step
Epoch 2/50
5478/5478 - 36s - loss: 40065.0625 - accuracy: 0.0000e+00 - 36s/epoch - 6ms/step
Epoch 3/50
5478/5478 - 34s - loss: 39227.5703 - accuracy: 0.0000e+00 - 34s/epoch - 6ms/step
Epoch 4/50
5478/5478 - 36s - loss: 37175.9375 - accuracy: 1.8255e-04 - 36s/epoch - 7ms/step
Epoch 5/50
5478/5478 - 34s - loss: 35693.0000 - accuracy: 5.4765e-04 - 34s/epoch - 6ms/step
Epoch 6/50
5478/5478 - 34s - loss: 34600.8984 - accuracy: 3.6510e-04 - 34s/epoch - 6ms/step
Epoch 7/50
5478/5478 - 35s - loss: 33739.0586 - accuracy: 3.6510e-04 - 35s/epoch - 6ms/step
Epoch 8/50
5478/5478 - 34s - loss: 32930.6133 - accuracy: 0.0000e+00 - 34s/epoch - 6ms/step
Epoch 9/50
5478/5478 - 35s - loss: 32508.3672 - accuracy: 1.8255e-04 - 35s/epoch - 6ms/step
Epoch 10/50
5478/5478 - 33s - loss: 31860.1680 - accuracy: 5.4765e-04 - 33s/epoch - 6ms/step
Epoch 11/50
5478/5478 - 35s - loss: 31224.0762 - accuracy: 5.4765e-04 - 35s/epo

In [20]:
train_score = model.evaluate(X, Y, verbose=0)
print('Train Score: %.2f MSE (%.2f RMSE)' % (train_score[0], np.sqrt(train_score[0] ** 2)))

Train Score: 23042.68 MSE (23042.68 RMSE)
